In [88]:
import pandas as pd
import dask.dataframe as dd
import json
import re
import numpy as np

In [89]:
df = pd.read_csv("./data/TSLA_2020_2022/features/TSLA_2020_2022_003.csv")
df["features"][0]


'[[-0.09616660106336444 -0.1312780345741137 -0.03355583488812036\n  -0.17013592425237037 -0.4804322624287979 -0.3626722446528914\n  -0.6784775217645621 -0.039478893969622515 -0.03777629695791152\n  0.4573223570190641]\n [10.398620611963786 9.154198538491375 -0.07457118011429675\n  0.5965288127590641 -0.6764925639982782 -0.05592429014938174\n  1.0272537733193632 -0.039478893969622515 -0.03777629695791152\n  0.0121317157712304]\n [-0.09616660106336444 -0.12400752719028164 0.009872177704301691\n  -0.1516906490330972 0.38223306447691524 -0.21919516640717396\n  0.0038149962690080027 -0.015719675058837312 -0.03777629695791152\n  0.41139514731369164]\n [-0.09616660106336444 0.00694150140423166 1.0280178062599743\n  0.4556673031616809 3.0617238525931456 1.078808299842274\n  -0.6784775217645621 -0.039478893969622515 -0.03777629695791152\n  0.5658578856152512]\n [-0.09616660106336444 -0.02677447789222021 -0.15418920320040386\n  0.013261285207784372 -0.807199431711265 0.25602372993357125\n  -0.33

In [90]:
def float_converter(x): return np.array(
    x.translate({ord(c): None for c in "[]"})
    .replace("\n", "")
    .replace("  ", " ")
     .split(" "),
    dtype=float
).reshape(10, -1)


def str_converter(x): return np.array(
    x.translate({ord(c): None for c in "'[]"})
    .replace("\n", "")
    .replace("  ", " ")
     .split(" "))



In [91]:
features_df = dd.read_csv(
    "./data/TSLA_2020_2022/features/" + "*.csv",
    converters={
        "features": float_converter,
        "inference": str_converter,
        "confidence": float_converter,
    }
)

In [92]:
from dask.diagnostics import ProgressBar
with ProgressBar():
    df = features_df.compute()

[########################################] | 100% Completed | 12.3s


In [93]:
df = df.set_index('date')
df

,Unnamed: 0,features,inference,confidence
date,,,,
2019-12-24,0,"[[-0.09616660106336444, -0.1308785561464306, -...","[LABEL_1, LABEL_0, LABEL_1, LABEL_1, LABEL_0, ...","[[0.9387555718421936], [0.5477803349494934], [..."
2019-12-24,1,"[[-0.09616660106336444, -0.12912085106462506, ...","[LABEL_0, LABEL_1, LABEL_1, LABEL_0, LABEL_1, ...","[[0.9979043006896972], [0.9986237287521362], [..."
2019-12-24,2,"[[-0.09616660106336444, -0.13159761731626016, ...","[LABEL_0, LABEL_0, LABEL_1, LABEL_1, LABEL_1, ...","[[0.7824580669403076], [0.9837669134140016], [..."
2019-12-24,3,"[[-0.09616660106336444, 0.012534199391794768, ...","[LABEL_0, LABEL_1, LABEL_0, LABEL_1, LABEL_0, ...","[[0.9160965085029602], [0.9771380424499512], [..."
2019-12-24,4,"[[-0.09616660106336444, -0.12760283303942935, ...","[LABEL_1, LABEL_1, LABEL_0, LABEL_1, LABEL_1, ...","[[0.998606503009796], [0.9978117346763612], [0..."
...,...,...,...,...
2022-03-01,9,"[[-0.09616660106336444, -0.13119813888857706, ...","[LABEL_0, LABEL_0, LABEL_1, LABEL_1, LABEL_0, ...","[[0.8168151378631592], [0.807685136795044], [0..."
2022-03-01,10,"[[-0.09616660106336444, -0.12968012086338138, ...","[LABEL_1, LABEL_1, LABEL_0, LABEL_1, LABEL_0, ...","[[0.98909991979599], [0.6970877647399902], [0...."
2022-03-01,11,"[[-0.09616660106336444, -0.13175740868733338, ...","[LABEL_1, LABEL_1, LABEL_1, LABEL_0, LABEL_1, ...","[[0.9985982775688172], [0.9861639142036438], [..."


In [94]:
df = df.explode(column=['features', 'inference', 'confidence'])
df['confidence'] = df['confidence'].apply(lambda x: x[0])


In [98]:
df.inference.value_counts()

LABEL_1      1290554
LABEL_0       697681
LABEL_1\n     143426
LABEL_0\n      77489
Name: inference, dtype: int64